In [2]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import os

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu,True)

In [29]:
import polars as pl
import numpy as np
import os
directory = '/app/finalDataSets'
# directory = r'D:\github\localt20'

playerStats = tf.convert_to_tensor(np.load(os.path.join(directory,'playerStats.npy')))

# Shapes of the data
playerStats_shape = playerStats.shape

playerStats_shape

TensorShape([1156, 22, 23])

In [30]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet

# Customize model input layer
cnnModel = models.Sequential([
    layers.Input(shape=(22, 23, 1)),  # Original input shape for player stats
    layers.Resizing(32, 32),          # Resize to fit MobileNet input size
    layers.Conv2D(3, (1, 1), activation='relu'),  # Convert 1 channel to 3 channels
    MobileNet(include_top=False, weights='imagenet', input_shape=(32, 32, 3)),
    layers.GlobalAveragePooling2D(),  # Optional: Pooling for fixed-size vector
    layers.Dense(50, activation='relu')  # Optional: Compress to desired number of features
])

cnnModel.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing_2 (Resizing)           │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 3)      │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenet_1.00_224 (Functional) │ (None, 1, 1, 1024)     │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │        51,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,280,120 (12.51 MB)

 Trainable params: 3,258,232 (12.43 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [33]:
pred = cnnModel.predict(tf.reshape(playerStats[500], (1, 22, 23)))
filtered_pred = pl.Series(pred[0]).filter(pl.Series(pred[0]) != 0)
filtered_pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


""
f32
0.080914
0.181773
0.057278
0.147138
0.269358
…
0.261064
0.159179
0.330841
